## 重回帰

## 目的
- アルバイトが好きか否かというデータと学問への興味の強さを数値化したデータから出席率を予測する

## データ
- N : データの個数
- A : アルバイトが好きか否か
- Score : 学問への興味の強さ
- Y : 一年の出席率

## パラメータ
- b1 : 切片
- b2 : Aに関する傾き
- b3 : Scoreに関する傾き
- sigma : 分散

## モデル

- $ mu[n] =b1 + b2*A[n] + b3*Score[n] $

- $Y[n] \sim Normal(\mu[n],sigma)$

In [1]:
%matplotlib inline

In [2]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [26]:
stan_code = """
data {
    int N;
    int<lower=0, upper=1> A[N];
    real<lower=0, upper=1> Score[N];
    real<lower=0, upper=1> Y[N];
}

parameters {
    real b1;
    real b2;
    real b3;
    real < lower = 0 > sigma;
}

transformed parameters {
    real mu[N];
    for (n in 1:N)
        mu[n] = b1 +b2 *A[n] + b3*Score[n];
}

model {
    for (n in 1:N)
        Y[n] ~ normal(mu[n],sigma);
}

generated quantities {
    real y_pred[N];
    for (n in 1:N)
        y_pred[n] = normal_rng(mu[n],sigma);
}

"""

In [27]:
df=pd.read_csv("data/data-attendance-1.txt")

stan_dat = {
    'N': len(df),
    'A': df["A"],
    'Y': df["Y"],
    'Score': df["Score"]/200,   
}

fit = pystan.stan(model_code = stan_code, data = stan_dat, iter = 1000, chains = 4)


In [45]:
la=fit.extract()

In [55]:
b1=np.median(la["b1"])
b2=np.median(la["b2"])
b3=np.median(la["b3"])
sigma=np.median(la["sigma"])
print(b1,b2,b3,sigma)

0.126476564983 -0.14374619578 0.321030141435 0.0512171850879


## モデリング結果
$\mu[n] = 0.126476564983 -0.14374619578 * A[n] +0.321030141435* Score[n] / 200$

$p(Y[n])=Normal(\mu[n],0.05)$